In [1]:
import datetime
import re
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from Project.Database import Db

In [2]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)
year2_minutes, meta = Db.load_data(hourly=False, meta=True, year=2)
year1_minutes

,Timestamp,DHW_ClothesWasherColdFlow,DHW_ClothesWasherHotFlow,DHW_DishwasherHotFlow,DHW_HeatPumpWaterHeaterEnergyTotal,DHW_HeatPumpWaterHeaterPowerTotal,DHW_ManifoldColdFlow,DHW_ManifoldHotFlow,DHW_MixValveColdFlow,DHW_RoomTempBasementHPWH,...,SHW_PSPSHW,SHW_SHWHX80galWaterFlow,Vent_HRVTempExhaustOut,Vent_HRVTempOutdoorin,Vent_HRVTempReturnIn,Vent_HRVTempSupplyOut,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,Load_StatusClothesWasher,Load_StatusDryerPowerTotal
0,2013-07-01 00:01:14-04:00,0.0,0.0,0.00000,0.000000,6.920000,0.000000,0.00000,0.00000,21.238651,...,0.000056,0.00000,24.712040,23.144780,22.552117,23.758415,125.4000,922.6568,0,0
1,2013-07-01 00:02:14-04:00,0.0,0.0,0.00000,0.132000,6.920000,0.000000,0.00000,0.00000,21.223797,...,0.000027,0.00000,24.710517,23.143295,22.523656,23.737489,127.0400,919.2424,0,0
2,2013-07-01 00:03:14-04:00,0.0,0.0,0.00000,0.264000,6.920000,0.000000,0.00000,0.00000,21.263438,...,0.000028,0.00000,24.691049,23.148787,22.539345,23.742217,153.3050,920.8184,0,0
3,2013-07-01 00:04:14-04:00,0.0,0.0,0.00000,0.396000,6.920000,0.000000,0.00000,0.00000,21.228323,...,0.000040,0.00000,24.686121,23.120120,22.539554,23.727526,143.4575,922.3944,0,0
4,2013-07-01 00:05:14-04:00,0.0,0.0,0.00000,0.528000,6.920000,0.000000,0.00000,0.00000,21.246059,...,0.000011,0.00000,24.683265,23.179975,22.506425,23.705258,141.8150,925.2832,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,2014-06-30 23:29:22-04:00,0.0,0.0,1.86261,1074.687075,765.920563,11.297412,57.77676,8.69218,19.589094,...,11.326110,1867.29955,25.037362,25.339981,23.912489,24.423000,110.6250,1517.2800,0,0
518788,2014-06-30 23:30:22-04:00,0.0,0.0,1.86261,1087.432650,766.929871,11.297412,57.77676,8.69218,19.521485,...,11.916921,1867.29955,25.033280,25.318893,23.898320,24.403229,124.5800,1394.6256,0,0
518789,2014-06-30 23:31:22-04:00,0.0,0.0,1.86261,1100.200137,768.525940,11.297412,57.77676,8.69218,19.619391,...,11.259323,1867.29955,24.992671,25.261438,23.849174,24.392863,131.1450,1311.6304,0,0
518790,2014-06-30 23:59:22-04:00,0.0,0.0,1.86261,1173.107485,6.920000,11.297412,57.77676,8.69218,21.028008,...,12.130983,1867.29955,24.985092,25.130462,24.119352,24.579783,127.0400,1325.5504,0,0


In [69]:
#Extract appliances from the meta data
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        (~self.index.str.contains("Child")) &
        (~self.index.str.contains("Prnt"))
))


appliance_location = pd.DataFrame(meta.loc[appliance_condition, "Measurement_Location"].sort_values(ascending=True))

appliance_location

,Measurement_Location
Unnamed: 0,
Load_StatusBA1Lights,Bath1
Load_StatusBA2Lights,Bath2
Load_StatusPlugLoadBR2Laptop,Bedroom2
Load_StatusBR2Lights,Bedroom2
Load_StatusPlugLoadBR3Laptop,Bedroom3
Load_StatusBR3Lights,Bedroom3
Load_StatusPlugLoadDesktopPCMonitor,Bedroom4
Load_StatusPlugLoadVacuum,Bedroom4
Load_StatusDRLights,Dining Room


In [65]:
#Combine minute data with the extracted appliances into a dataframe
minute_appliances_status = year1_minutes[["Timestamp"] + appliance_location.index.tolist()].copy()
minute_appliances_status

,Timestamp,Load_StatusBA1Lights,Load_StatusBA2Lights,Load_StatusPlugLoadBR2Laptop,Load_StatusBR2Lights,Load_StatusPlugLoadBR3Laptop,Load_StatusBR3Lights,Load_StatusPlugLoadDesktopPCMonitor,Load_StatusPlugLoadVacuum,Load_StatusDRLights,...,Load_StatusMBALights,Load_StatusMBRLights1,Load_StatusMBRLights2,Load_StatusPlugLoadMBRTV,Load_StatusPlugLoadMBRBlueRay,Load_StatusPlugLoadFan,Load_StatusPlugLoadHeatingPad,Load_StatusPlugLoadIron,Load_StatusClothesWasher,Load_StatusDryerPowerTotal
0,2013-07-01 00:01:14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,2013-07-01 00:02:14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,2013-07-01 00:03:14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,2013-07-01 00:04:14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,2013-07-01 00:05:14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,2014-06-30 23:29:22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
518788,2014-06-30 23:30:22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
518789,2014-06-30 23:31:22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
518790,2014-06-30 23:59:22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [71]:
appliance_job_list = []

for appliance, appliance_row in appliance_location.iterrows():
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load_Status")[-1]))
    appliance_switch = minute_appliances_status[(minute_appliances_status[appliance] != minute_appliances_status[appliance].shift(1))][["Timestamp", appliance]][1:]
    appliance_switch.reset_index(inplace=True, drop=True)
    for index, row in appliance_switch.iterrows():
        if row[appliance]:
            try:
                appliance_job_list.append({"appliance": name, "start": row["Timestamp"], "end": appliance_switch.loc[index+1, "Timestamp"], "location": appliance_row["Measurement_Location"]})
            except:
                continue

appliance_job_list

[{'appliance': 'B A1 Lights',
  'start': Timestamp('2013-07-01 08:13:17'),
  'end': Timestamp('2013-07-01 08:18:17'),
  'location': 'Bath1'},
 {'appliance': 'B A1 Lights',
  'start': Timestamp('2013-07-01 18:00:31'),
  'end': Timestamp('2013-07-01 18:04:31'),
  'location': 'Bath1'},
 {'appliance': 'B A1 Lights',
  'start': Timestamp('2013-07-02 15:59:28'),
  'end': Timestamp('2013-07-02 16:04:28'),
  'location': 'Bath1'},
 {'appliance': 'B A1 Lights',
  'start': Timestamp('2013-07-02 18:00:28'),
  'end': Timestamp('2013-07-02 18:04:28'),
  'location': 'Bath1'},
 {'appliance': 'B A1 Lights',
  'start': Timestamp('2013-07-03 07:38:28'),
  'end': Timestamp('2013-07-03 07:43:28'),
  'location': 'Bath1'},
 {'appliance': 'B A1 Lights',
  'start': Timestamp('2013-07-03 21:36:27'),
  'end': Timestamp('2013-07-03 21:41:27'),
  'location': 'Bath1'},
 {'appliance': 'B A1 Lights',
  'start': Timestamp('2013-07-04 15:59:26'),
  'end': Timestamp('2013-07-04 16:04:26'),
  'location': 'Bath1'},
 {'app

In [72]:
fig = px.timeline(appliance_job_list, x_start="start", x_end="end", y="appliance", color="location")
fig.write_html(Db.get_save_file_directory("Appliance_status_gantt.html"))


In [74]:
residens_condition = (lambda self: (
    (self["Subsystem"] == "Loads") &
    (self["Parameter"] == "Status_OnOff") &
    (self.index.str.contains("Sens"))
    )
)

residens_location = meta.loc[residens_condition].index.tolist()
residens_location.sort()

residens_location

['Load_StatusSensHeatChildADOWN',
 'Load_StatusSensHeatChildAUP',
 'Load_StatusSensHeatChildBDOWN',
 'Load_StatusSensHeatChildBUP',
 'Load_StatusSensHeatPrntADOWN',
 'Load_StatusSensHeatPrntAUP',
 'Load_StatusSensHeatPrntBDOWN',
 'Load_StatusSensHeatPrntBUP']

In [75]:
#Combine minute data with the extracted residens into a dataframe
residens_location_status = year1_minutes[["Timestamp"] + residens_location].copy()

In [77]:
person_status_list = []

for col in residens_location:
    name = "".join(re.findall("[PC][a-z]+[AB]", col))
    person_location = residens_location_status[lambda self: (self[col] != self[col].shift(1))][
                           ["Timestamp", col]][1:]

    df_indices = person_location.index.tolist()
    for index, df_index in enumerate(df_indices):
        if person_location.loc[df_index, col]:
            try:
                person_status_list.append({"appliance": name, "start": person_location.loc[df_index, "Timestamp"], "end": person_location.loc[df_indices[index+1], "Timestamp"], "location": {"DOWN": "First floor", "UP": "Second floor"}["".join(col.split(name)[-1])]})
            except:
                continue
person_status_list

[{'appliance': 'ChildA',
  'start': Timestamp('2013-07-01 06:58:17'),
  'end': Timestamp('2013-07-01 07:58:17'),
  'location': 'First floor'},
 {'appliance': 'ChildA',
  'start': Timestamp('2013-07-01 19:28:31'),
  'end': Timestamp('2013-07-01 20:13:31'),
  'location': 'First floor'},
 {'appliance': 'ChildA',
  'start': Timestamp('2013-07-02 06:58:29'),
  'end': Timestamp('2013-07-02 07:58:29'),
  'location': 'First floor'},
 {'appliance': 'ChildA',
  'start': Timestamp('2013-07-02 15:58:28'),
  'end': Timestamp('2013-07-02 16:28:28'),
  'location': 'First floor'},
 {'appliance': 'ChildA',
  'start': Timestamp('2013-07-02 17:58:28'),
  'end': Timestamp('2013-07-02 18:28:28'),
  'location': 'First floor'},
 {'appliance': 'ChildA',
  'start': Timestamp('2013-07-02 19:28:28'),
  'end': Timestamp('2013-07-02 20:13:28'),
  'location': 'First floor'},
 {'appliance': 'ChildA',
  'start': Timestamp('2013-07-03 06:58:28'),
  'end': Timestamp('2013-07-03 07:58:28'),
  'location': 'First floor'},

In [78]:
fig = px.timeline(person_status_list, x_start="start", x_end="end", y="appliance", color="location")
fig.write_html(Db.get_save_file_directory("Person_status_gantt.html"))
# fig.show()

In [79]:
fig = px.timeline(person_status_list+appliance_job_list, x_start="start", x_end="end", y="appliance", color="location")
# fig.show()

In [80]:
fig.write_html(Db.get_save_file_directory("Person_appliance_status_gantt.html"))